<a href="https://colab.research.google.com/github/Joshua-K-Bowles/Vivino-Webscrape-and-Data-Analysis/blob/main/Vivino_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Vivino Data Cleaning

For easy of use and replication, all modules have been added at the top. 

Introduction:

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from googletrans import Translator

### Import Vivino Wine Data from Vivino Scrape

In [ ]:
df = pd.read_csv(r'/vivino_all.csv')

#### Drop Duplicates from Data frame

In [ ]:
df_vino = df.drop_duplicates(keep='first')
df_vino

,Winery,Year,Wine ID,Wine,Rating,num_review,price,Country,Region,most_used_grape,type,natural,sourcefilename
0,Baron de Ley,2020,8729994,Rosado de Lágrima 2020,4.2,42,7.150000,Argentina,La Rioja,Malbec,4,False,C:\SD card\Documents\Data Analytics\Vivino Web...
1,New Age,N.V.,7704484,Sweet White N.V.,3.9,5672,7.430000,Argentina,Mendoza,Malbec,2,False,C:\SD card\Documents\Data Analytics\Vivino Web...
2,Alamos,2018,68764,Malbec Rosé 2018,3.9,384,8.000000,Argentina,Valle de Uco,Malbec,4,False,C:\SD card\Documents\Data Analytics\Vivino Web...
3,Alamos,2019,2166,Torrontes 2019,3.8,820,8.000000,Argentina,Salta,Malbec,2,False,C:\SD card\Documents\Data Analytics\Vivino Web...
4,Alamos,2020,1387085,Cabernet Sauvignon 2020,3.6,489,8.000000,Argentina,Mendoza,Malbec,1,False,C:\SD card\Documents\Data Analytics\Vivino Web...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40989,Glenelly,2017,1170241,The Glass Collection Syrah 2017,3.6,199,10.890000,Sudáfrica,Stellenbosch,Cabernet Sauvignon,1,False,C:\SD card\Documents\Data Analytics\Vivino Web...
40990,Lourensford,2019,1176448,The River Garden Sauvignon Blanc 2019,3.4,107,7.900000,Sudáfrica,Western Cape,Cabernet Sauvignon,2,False,C:\SD card\Documents\Data Analytics\Vivino Web...
40991,Springfield Estate,2021,1192074,Life from Stone Sauvignon Blanc 2021,4.3,323,12.800000,Sudáfrica,Robertson,Cabernet Sauvignon,2,False,C:\SD card\Documents\Data Analytics\Vivino Web...
40992,Meerlust,2017,19306,Cabernet Sauvignon 2017,4.1,178,22.900000,Sudáfrica,Stellenbosch,Cabernet Sauvignon,1,False,C:\SD card\Documents\Data Analytics\Vivino Web...


#### Fix inconsistencies with Country Names: 
The Vivino Web Scrape returned some information in Spanish which makes sense since I live in Spain and have some confusing Spanish settings on computer and web browser. However, the data I scraped from wikipedia was in English. So here I need to translate the 'Country' names to English. 

First, to save time and resources we will need to get a list of unique country names from our data frame.

In [ ]:
df_country_unique = pd.DataFrame(df_vino['Country'].unique())
df_country_unique.columns =['country']
df_country_unique

,country
0,Argentina
1,Australia
2,Chile
3,Alemania
4,España
5,Francia
6,Italia
7,Portugal
8,Estados Unidos
9,Sudáfrica


Now we can translate the list using Google's Translator module. As a side note, I initially did this exercise in my Jupyter Notebook without success. However, after some thought and trial and error, I decided that Google Colab might have less problems connecting to and working with Google's Translate module... I was right. 

In [ ]:
translator = Translator()  
df_country_unique['Country'] = df_country_unique['country'].apply(lambda x: translator.translate(x, dest='en').text)
df_country_unique

,country,Country
0,Argentina,Argentina
1,Australia,Australia
2,Chile,Chile
3,Alemania,Germany
4,España,Spain
5,Francia,France
6,Italia,Italy
7,Portugal,Portugal
8,Estados Unidos,United States
9,Sudáfrica,South Africa


Now that we have a list of unique names in English, it is time to change our Dataframe to reflect them. 

In [ ]:
df_vino = pd.merge(left=df_vino, right=df_country_unique, how='left', left_on='Country', right_on='country')
df_vino

,Winery,Year,Wine ID,Wine,Rating,num_review,price,Country_x,Region,most_used_grape,type,natural,sourcefilename,country,Country_y
0,Baron de Ley,2020,8729994,Rosado de Lágrima 2020,4.2,42,7.150000,Argentina,La Rioja,Malbec,4,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Argentina,Argentina
1,New Age,N.V.,7704484,Sweet White N.V.,3.9,5672,7.430000,Argentina,Mendoza,Malbec,2,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Argentina,Argentina
2,Alamos,2018,68764,Malbec Rosé 2018,3.9,384,8.000000,Argentina,Valle de Uco,Malbec,4,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Argentina,Argentina
3,Alamos,2019,2166,Torrontes 2019,3.8,820,8.000000,Argentina,Salta,Malbec,2,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Argentina,Argentina
4,Alamos,2020,1387085,Cabernet Sauvignon 2020,3.6,489,8.000000,Argentina,Mendoza,Malbec,1,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Argentina,Argentina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12200,Glenelly,2017,1170241,The Glass Collection Syrah 2017,3.6,199,10.890000,Sudáfrica,Stellenbosch,Cabernet Sauvignon,1,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Sudáfrica,South Africa
12201,Lourensford,2019,1176448,The River Garden Sauvignon Blanc 2019,3.4,107,7.900000,Sudáfrica,Western Cape,Cabernet Sauvignon,2,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Sudáfrica,South Africa
12202,Springfield Estate,2021,1192074,Life from Stone Sauvignon Blanc 2021,4.3,323,12.800000,Sudáfrica,Robertson,Cabernet Sauvignon,2,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Sudáfrica,South Africa
12203,Meerlust,2017,19306,Cabernet Sauvignon 2017,4.1,178,22.900000,Sudáfrica,Stellenbosch,Cabernet Sauvignon,1,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Sudáfrica,South Africa


Time to change the 'type' column to reflect the style of wine it represents by creating a new data frame to reference. The 'Style' data was collected from Vivino's website, and I decided it easy enough and small enough to create a data frame. 

In [ ]:
type = {"Type": [1,2,3,4,7,24], 'Style': ['Red', 'White', 'Sparkling', 'Rose', 'Dessert', 'Fortified']}

In [ ]:
df_type = pd.DataFrame(type)
df_type

,Type,Style
0,1,Red
1,2,White
2,3,Sparkling
3,4,Rose
4,7,Dessert
5,24,Fortified


Once again we can merge the data frames into the df_vino data frame. This time, I chose to name it 'Style'.

In [ ]:
df_vino = pd.merge(left=df_vino, right=df_type, how='left', left_on='type', right_on='Type')
df_vino

,Winery,Year,Wine ID,Wine,Rating,num_review,price,Country_x,Region,most_used_grape,type,natural,sourcefilename,country,Country_y,Type,Style
0,Baron de Ley,2020,8729994,Rosado de Lágrima 2020,4.2,42,7.150000,Argentina,La Rioja,Malbec,4,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Argentina,Argentina,4,Rose
1,New Age,N.V.,7704484,Sweet White N.V.,3.9,5672,7.430000,Argentina,Mendoza,Malbec,2,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Argentina,Argentina,2,White
2,Alamos,2018,68764,Malbec Rosé 2018,3.9,384,8.000000,Argentina,Valle de Uco,Malbec,4,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Argentina,Argentina,4,Rose
3,Alamos,2019,2166,Torrontes 2019,3.8,820,8.000000,Argentina,Salta,Malbec,2,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Argentina,Argentina,2,White
4,Alamos,2020,1387085,Cabernet Sauvignon 2020,3.6,489,8.000000,Argentina,Mendoza,Malbec,1,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Argentina,Argentina,1,Red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12200,Glenelly,2017,1170241,The Glass Collection Syrah 2017,3.6,199,10.890000,Sudáfrica,Stellenbosch,Cabernet Sauvignon,1,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Sudáfrica,South Africa,1,Red
12201,Lourensford,2019,1176448,The River Garden Sauvignon Blanc 2019,3.4,107,7.900000,Sudáfrica,Western Cape,Cabernet Sauvignon,2,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Sudáfrica,South Africa,2,White
12202,Springfield Estate,2021,1192074,Life from Stone Sauvignon Blanc 2021,4.3,323,12.800000,Sudáfrica,Robertson,Cabernet Sauvignon,2,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Sudáfrica,South Africa,2,White
12203,Meerlust,2017,19306,Cabernet Sauvignon 2017,4.1,178,22.900000,Sudáfrica,Stellenbosch,Cabernet Sauvignon,1,False,C:\SD card\Documents\Data Analytics\Vivino Web...,Sudáfrica,South Africa,1,Red


However, Now that we have merged the data frames, we need to clean up the 'df_vino' data frame. 

#### Drop Columns
Drop the extra and unnecessary column names.

In [ ]:
df_vino.drop(['country', 'Country_x', 'sourcefilename', 'Type', 'type', ], inplace=True, axis=1)

Rename the rest of the columns for consistency.

In [ ]:
df_vino.rename(columns = {'Country_y':'Country', 'num_review':'Reviews',
                              'price':'Price', 'most_used_grape':'Primary_Grape','natural':'Natural','Wine ID':'Wine_ID'}, inplace = True)

### I also want to scrape ISO Country Codes for each country to create maps of the data. 

In [ ]:
url = 'https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3'
data = requests.get(url)
soup= BeautifulSoup(data.content, "html.parser")

In [ ]:
list_soup = soup.find_all("div", {"class": "div-col"})[0].find_all('li')

df_country_code = pd.DataFrame([[item.find_next("a").get("title"),
                   item.find_next("span").get_text()]
                for item in list_soup if item.find_next("i")],
                columns=("Country","Country_Code"))

print(df_country_code.tail())

          Country Country_Code
244         Samoa          WSM
245         Yemen          YEM
246  South Africa          ZAF
247        Zambia          ZMB
248      Zimbabwe          ZWE


In [ ]:
df_vino = df_vino.merge(df_country_code, on='Country', how='left')
df_vino

,Winery,Year,Wine_ID,Wine,Rating,Reviews,Price,Region,Primary_Grape,Natural,Country,Style,Country_Code
0,Baron de Ley,2020,8729994,Rosado de Lágrima 2020,4.2,42,7.150000,La Rioja,Malbec,False,Argentina,Rose,ARG
1,New Age,N.V.,7704484,Sweet White N.V.,3.9,5672,7.430000,Mendoza,Malbec,False,Argentina,White,ARG
2,Alamos,2018,68764,Malbec Rosé 2018,3.9,384,8.000000,Valle de Uco,Malbec,False,Argentina,Rose,ARG
3,Alamos,2019,2166,Torrontes 2019,3.8,820,8.000000,Salta,Malbec,False,Argentina,White,ARG
4,Alamos,2020,1387085,Cabernet Sauvignon 2020,3.6,489,8.000000,Mendoza,Malbec,False,Argentina,Red,ARG
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12200,Glenelly,2017,1170241,The Glass Collection Syrah 2017,3.6,199,10.890000,Stellenbosch,Cabernet Sauvignon,False,South Africa,Red,ZAF
12201,Lourensford,2019,1176448,The River Garden Sauvignon Blanc 2019,3.4,107,7.900000,Western Cape,Cabernet Sauvignon,False,South Africa,White,ZAF
12202,Springfield Estate,2021,1192074,Life from Stone Sauvignon Blanc 2021,4.3,323,12.800000,Robertson,Cabernet Sauvignon,False,South Africa,White,ZAF
12203,Meerlust,2017,19306,Cabernet Sauvignon 2017,4.1,178,22.900000,Stellenbosch,Cabernet Sauvignon,False,South Africa,Red,ZAF


## Clean and pretty data ready for csv export! 



In [ ]:
# Set up connection to Google Drive
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
# Saves the file to CSV
df_vino.to_csv("vivino_clean.csv", index=False)
!cp vivino_clean.csv "drive/My Drive/"